In [1]:
import io
import pandas as pd
import requests
import re
import xml.etree.ElementTree as ET
import subprocess

from bs4 import BeautifulSoup

In [2]:
from platform import python_version
print(python_version())

3.6.9


In [3]:
import sys
sys.version

'3.6.9 (default, Nov  7 2019, 10:44:02) \n[GCC 8.3.0]'

In [14]:
tree = ET.parse('ABSA16FR_Museums_Gold-withcontent.xml')
root = tree.getroot()

In [15]:
links = list()

In [6]:
f = open("ABSA16FR_Restaurants_index.txt", "r")
for x in f:
    val = x.split('\t')
    val[2] = val[2].replace('\n', '')
    links.append(val)

In [7]:
dataL = pd.DataFrame(links, columns=['id', 'link', 'nbSentence'])

In [8]:
dataL = dataL.set_index('id')
dataL.head()

,link,nbSentence
id,,
g1079435-d3498474-r271346275,https://www.tripadvisor.fr/ShowUserReviews-g10...,1
g1080298-d1332050-r275655485,https://www.tripadvisor.fr/ShowUserReviews-g10...,2
g1080298-d4701165-r281527562,https://www.tripadvisor.fr/ShowUserReviews-g10...,3
g1080559-d3686505-r151173923,https://www.tripadvisor.fr/ShowUserReviews-g10...,1
g1080574-d1337519-r211045323,https://www.tripadvisor.fr/ShowUserReviews-g10...,6


In [9]:
total_review = len(dataL)
total_sentence = pd.to_numeric(dataL['nbSentence']).sum()
total_review, total_sentence

(337, 1704)

In [10]:
dataL.index

Index(['g1079435-d3498474-r271346275', 'g1080298-d1332050-r275655485',
       'g1080298-d4701165-r281527562', 'g1080559-d3686505-r151173923',
       'g1080574-d1337519-r211045323', 'g1582538-d3877732-r250915172',
       'g1833139-d7211477-r282922737', 'g187073-d1328515-r282975754',
       'g187073-d2234111-r233672137', 'g187073-d2346663-r285268023',
       ...
       'g3463365-d824605-r268214202', 'g608761-d1334387-r277341177',
       'g608761-d1334432-r281522672', 'g608761-d2053817-r143008138',
       'g608761-d2445738-r244940036', 'g608761-d3778492-r181222052',
       'g608761-d4750470-r279534176', 'g608761-d5960507-r277194025',
       'g608761-d7105002-r282300173', 'g635821-d5792203-r217535741'],
      dtype='object', name='id', length=337)

In [11]:
r = requests.get(dataL.loc['g1079435-d3498474-r271346275']['link'])

In [12]:
m = re.search('<p class="partial_entry">(.)*[a-zA-Z0-9]<\/p>', r.text)
if m:
    found = m.group(0)

In [13]:
len('<p class="partial_entry">')
s = "abcdefghijklmnopq"
s[5:-2]

'fghijklmno'

In [6]:
data = list()

In [7]:
!ls

 ABSA16FR-download.jar			      en-sent.bin
 ABSA16FR_Museums_Gold-withcontent.xml	      export_data.csv
 ABSA16FR_Restaurants_Gold-withcontent.xml    fr-sent.bin
 ABSA16FR_Restaurants_guidelines.pdf	     'Get Data.ipynb'
 ABSA16FR_Restaurants_index.txt		      input.txt
 ABSA16FR_Restaurants_Train-withcontent.xml   opennlp-python
 ABSA16FR_Restaurants_Train.xml		      README.txt
 apache-opennlp-1.9.2			      sentences.txt
 Data2LCF-ATEPC.ipynb			      ter.atepc.train.dat


In [85]:
l = len(root)
i = 0
parsing_stats = 0
parsing_stats_sent = 0
opinions_stats = 0
opinions_stats_sent = 0
notfound_stats = 0
notfound_stats_sent = 0
for review in root:
    id = review.attrib['rid']
    #r = requests.get(dataL.loc[id]['link'])
    doc = BeautifulSoup(r.text)
    doc = doc.find('p', attrs={'class' : 'partial_entry'})
    if doc != None:
        doc = doc.get_text()
   
        
        with open("input.txt", "w", encoding="utf-8") as f: # or "rb"
            f.write(doc)
            f.close()
        
        subprocess.call("sh apache-opennlp-1.9.2/bin/opennlp SentenceDetector ./fr-sent.bin < ./input.txt > ./sentences.txt", shell=True) 
        
        with open('./sentences.txt', 'r') as content_file:
            content = content_file.read()
            index_sentence = 0
            sentences = content.split('\n')[:-2]
            if len(review[0]) == len(sentences):
                goodSentence = 0
                for sentence in sentences:
                    opinions = review[0][index_sentence][1]
                    goodOpinion = 0
                    for opinion in opinions:
                        if(sentence[int(opinion.attrib['from']):int(opinion.attrib['to'])] == opinion.attrib['target'] or opinion.attrib['target'] == "NULL"):
                            goodOpinion += 1
                    if(goodOpinion == len(opinions)):
                        goodSentence += 1
                    index_sentence+=1
                    
                index_sentence = 0   
                if(goodSentence == len(review[0])):
                    for sentence in sentences:
                        opinions = review[0][index_sentence][1]
                        for opinion in opinions:
                            data.append([id, str(index_sentence), doc, sentence, opinion.attrib['target'], opinion.attrib['category'], opinion.attrib['polarity'], opinion.attrib['from'], opinion.attrib['to']])
                        if(len(opinions) == 0):
                            data.append([id, str(index_sentence), doc, sentence, "NULL", "NULL", "NULL", 0, 0])
                        index_sentence += 1
                else:
                    for sentence in sentences:
                        opinions = review[0][index_sentence][1]
                        opinions_stats_sent += len(opinions)
                        index_sentence += 1
                        
                    opinions_stats+=1
                    print("Parsing error on ressource id {} due to bad opennlp version. Opinions don't corresponds with the sentences detected".format(id))
            else:
                for sentence in sentences:
                    opinions = review[0][index_sentence][1]
                    opinions_stats_sent += len(opinions)
                    index_sentence += 1
                    
                parsing_stats+=1
                print("Parsing error on ressource id {} due to bad opennlp version. Bad number of sentences detected".format(id))
            
        print(str(i)+"/"+str(l)+" "+id+" Not Found : {} | Parsing : {} | Lost review (%) : {}".format(notfound_stats, parsing_stats+opinions_stats, (parsing_stats+opinions_stats+notfound_stats)*100/(i+1)), end="\r")
        
    else:
        notfound_stats += 1
        print("Ressource {} doesn't contains a review".format(id))
    
    i += 1     

Parsing error on ressource id g1080298-d4701165-r281527562 due to bad opennlp version. Bad number of sentences detected
Ressource g1080574-d1337519-r211045323 doesn't contains a review Lost review (%) : 25.033333333333336
Parsing error on ressource id g1833139-d7211477-r282922737 due to bad opennlp version. Bad number of sentences detected
Ressource g187073-d1328515-r282975754 doesn't contains a review| Lost review (%) : 42.857142857142854
Ressource g187073-d2234111-r233672137 doesn't contains a review
Parsing error on ressource id g187073-d2346663-r285268023 due to bad opennlp version. Bad number of sentences detected
Parsing error on ressource id g187075-d1324447-r268262547 due to bad opennlp version. Bad number of sentences detected
Parsing error on ressource id g187075-d3489527-r254992608 due to bad opennlp version. Bad number of sentences detected
Ressource g187077-d2250282-r285122402 doesn't contains a review| Lost review (%) : 53.333333333333336
Parsing error on ressource id g18

Parsing error on ressource id g187253-d1098498-r217655976 due to bad opennlp version. Bad number of sentences detected
Ressource g187253-d1223545-r227663690 doesn't contains a review85 | Lost review (%) : 65.0
Parsing error on ressource id g187253-d1328827-r266271011 due to bad opennlp version. Bad number of sentences detected
Parsing error on ressource id g187253-d1331961-r118398179 due to bad opennlp version. Bad number of sentences detected
Parsing error on ressource id g187253-d1332190-r282285117 due to bad opennlp version. Bad number of sentences detected
Ressource g187253-d1639457-r152371370 doesn't contains a review88 | Lost review (%) : 63.745019920318725
Parsing error on ressource id g187253-d1854779-r278211239 due to bad opennlp version. Bad number of sentences detected
Parsing error on ressource id g187253-d2027102-r115708876 due to bad opennlp version. Bad number of sentences detected
Parsing error on ressource id g187253-d2161275-r243891446 due to bad opennlp version. Bad 

In [86]:
print("Ressources not found (404) : "+str(notfound_stats)+" --> "+str(notfound_stats*100/total_review))
print("Ressources pass due to SentenceDetector : "+str(parsing_stats+opinions_stats)+" --> "+str((parsing_stats+opinions_stats)*100/total_review))
print("Total : "+str(total_review))
print("Total : "+str(total_review-(parsing_stats+opinions_stats+notfound_stats))+" --> "+str(100-(parsing_stats+opinions_stats+notfound_stats)*100/total_review))

Ressources not found (404) : 89 --> 26.409495548961424
Ressources pass due to SentenceDetector : 124 --> 36.795252225519285
Total : 337
Total : 124 --> 36.795252225519285


In [87]:
dataDf = pd.DataFrame(data, columns=['id', 'index_sentence', 'review', 'sentence', 'target', 'category', 'polarity', 'from', 'to'])

In [88]:
data[0][2]

"Un service passable .. Des plats surcuits, des sauces figées dans leur pot (on se demande si elles n'étaient pas sur d'autres tables avant de nous être servies) , le prix est correct mais le goût n'est pas au rendez vous ... très déçu par ce resto"

In [89]:
dataDf

,id,index_sentence,review,sentence,target,category,polarity,from,to
0,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",service,SERVICE#GENERAL,neutral,3,10
1,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",plats,FOOD#QUALITY,negative,27,32
2,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",sauces,FOOD#QUALITY,negative,47,53
3,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",NULL,RESTAURANT#PRICES,neutral,0,0
4,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",NULL,FOOD#QUALITY,negative,0,0
...,...,...,...,...,...,...,...,...,...
873,g3463365-d824605-r268214202,0,Nous venons assez régulièrement dans ce restau...,Nous venons assez régulièrement dans ce restau...,NULL,SERVICE#GENERAL,negative,0,0
874,g608761-d1334432-r281522672,0,Un petit repas en famille. Déjà visité il y a ...,Un petit repas en famille.,NULL,NULL,NULL,0,0
875,g608761-d1334432-r281522672,1,Un petit repas en famille. Déjà visité il y a ...,"Déjà visité il y a quelques années, mais un tr...",NULL,RESTAURANT#GENERAL,positive,0,0
876,g608761-d1334432-r281522672,2,Un petit repas en famille. Déjà visité il y a ...,Salle agréable.,Salle,AMBIENCE#GENERAL,positive,0,5


In [90]:
nul = pd.isnull(dataDf)
print(nul[nul['sentence']].index.values)

[]


In [91]:
dataDf.to_csv ('./export_data.csv', header=True)

In [82]:
dataDf2 = pd.read_csv('export_data.csv')

In [83]:
dataDf2

,Unnamed: 0,id,index_sentence,review,sentence,target,category,polarity,from,to
0,0,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",service,SERVICE#GENERAL,neutral,3.0,10.0
1,1,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",plats,FOOD#QUALITY,negative,27.0,32.0
2,2,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",sauces,FOOD#QUALITY,negative,47.0,53.0
3,3,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",NaN,RESTAURANT#PRICES,neutral,0.0,0.0
4,4,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",NaN,FOOD#QUALITY,negative,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
873,873,g3463365-d824605-r268214202,0,Nous venons assez régulièrement dans ce restau...,Nous venons assez régulièrement dans ce restau...,NaN,SERVICE#GENERAL,negative,0.0,0.0
874,874,g608761-d1334432-r281522672,0,Un petit repas en famille. Déjà visité il y a ...,Un petit repas en famille.,NaN,NaN,NaN,NaN,NaN
875,875,g608761-d1334432-r281522672,1,Un petit repas en famille. Déjà visité il y a ...,"Déjà visité il y a quelques années, mais un tr...",NaN,RESTAURANT#GENERAL,positive,0.0,0.0
876,876,g608761-d1334432-r281522672,2,Un petit repas en famille. Déjà visité il y a ...,Salle agréable.,Salle,AMBIENCE#GENERAL,positive,0.0,5.0


In [68]:
nul = pd.isnull(dataDf2)
for v in nul[nul['sentence']].index.values:
    dataDf = dataDf[dataDf.id != dataDf.iloc[v].id]

In [69]:
dataDf.iloc[200]

id                                      g187147-d2146860-r282299845
index_sentence                                                    0
review            Un petit bistrot de quartier sympa, on y mange...
sentence          Un petit bistrot de quartier sympa, on y mange...
target                                                      service
category                                            SERVICE#GENERAL
polarity                                                   positive
from                                                             58
to                                                               65
Name: 200, dtype: object

In [70]:
dataDf.iloc[202]

id                                      g187147-d2146860-r282299845
index_sentence                                                    1
review            Un petit bistrot de quartier sympa, on y mange...
sentence          Plats de bistrot classiques, pour des prix "pa...
target                                                         NULL
category                                                FOOD#PRICES
polarity                                                    neutral
from                                                              0
to                                                                0
Name: 202, dtype: object

In [71]:
dataDf3 = dataDf2[dataDf2.sentence.notnull()]

In [72]:
dataDf3

,Unnamed: 0,id,index_sentence,review,sentence,target,category,polarity,from,to
0,0,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",service,SERVICE#GENERAL,neutral,3,10
1,1,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",plats,FOOD#QUALITY,negative,27,32
2,2,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",sauces,FOOD#QUALITY,negative,47,53
3,3,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",NaN,RESTAURANT#PRICES,neutral,0,0
4,4,g1079435-d3498474-r271346275,0,"Un service passable .. Des plats surcuits, des...","Un service passable .. Des plats surcuits, des...",NaN,FOOD#QUALITY,negative,0,0
...,...,...,...,...,...,...,...,...,...,...
844,844,g3463365-d824605-r268214202,0,Nous venons assez régulièrement dans ce restau...,Nous venons assez régulièrement dans ce restau...,plats,FOOD#STYLE_OPTIONS,positive,57,62
845,845,g3463365-d824605-r268214202,0,Nous venons assez régulièrement dans ce restau...,Nous venons assez régulièrement dans ce restau...,NaN,SERVICE#GENERAL,negative,0,0
846,846,g608761-d1334432-r281522672,1,Un petit repas en famille. Déjà visité il y a ...,"Déjà visité il y a quelques années, mais un tr...",NaN,RESTAURANT#GENERAL,positive,0,0
847,847,g608761-d1334432-r281522672,2,Un petit repas en famille. Déjà visité il y a ...,Salle agréable.,Salle,AMBIENCE#GENERAL,positive,0,5


In [32]:
"Un service passable"[3:10]

'service'

In [16]:
data = list()

In [17]:
for review in root:
    id = review.attrib['rid']
    index_sentence = 0
    for sentence in review[0]:
        for opinion in sentence[1]:
            data.append([id, str(index_sentence), sentence[0].text, opinion.attrib['target'], opinion.attrib['category'], opinion.attrib['polarity'], opinion.attrib['from'], opinion.attrib['to']])
        index_sentence+=1

IndexError: child index out of range

In [10]:
dataDf = pd.DataFrame(data, columns=['id', 'index_sentence', 'sentence', 'target', 'category', 'polarity', 'from', 'to'])

In [11]:
dataDf

,id,index_sentence,sentence,target,category,polarity,from,to
0,g1368199-d3726982-r258755756,0,Après le palais du facteur nous voici à la hal...,restaurant,RESTAURANT#GENERAL,neutral,60,70
1,g1368199-d3726982-r258755756,1,Le patron nous accueille très chaleureusement ...,patron,SERVICE#GENERAL,positive,3,9
2,g1368199-d3726982-r258755756,2,Tout n'est pas fait maison mais c'est bon.,NULL,FOOD#QUALITY,positive,0,0
3,g1368199-d3726982-r258755756,3,Le prix est un peu élevé pour un repas qui n'a...,repas,FOOD#PRICES,negative,33,38
4,g1368199-d4713084-r245661593,0,Des pizzas locales que vous ne mangerez pas ai...,pizzas,FOOD#STYLE_OPTIONS,positive,4,10
...,...,...,...,...,...,...,...,...
949,g660465-d3531828-r267383919,3,Dommage pour ce petit restaurant sympa.,restaurant,RESTAURANT#GENERAL,positive,22,32
950,g660465-d5932339-r281681321,0,Le repas est juste indécent!,repas,FOOD#QUALITY,negative,3,8
951,g660465-d5932339-r281681321,1,Rien de bon même la pomme de terre n'était pas...,pomme de terre,FOOD#QUALITY,negative,20,34
952,g660465-d5932339-r281681321,2,"Ne surtout pas y aller pour bien manger, par c...",ambiance,AMBIENCE#GENERAL,positive,87,95


In [12]:
nul = pd.isnull(dataDf)
print(nul[nul['sentence']].index.values)

[]


In [13]:
dataDf.to_csv ('./export_data_restaurants_test.csv', header=True)